In [ ]:
import os
import pandas as pd
import numpy as np
import re
import nltk
import unicodedata
import nltk
import gensim
import math
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
nltk.download('stopwords')
import joblib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df1 = pd.read_csv('/content/question-answer1.csv', sep=',')
#frames = [df1]
#df = pd.concat(frames)

df2['ArticleText'] = pd.read_csv('/content/train-w.txt')



#def getArticleText(file):
  #f = open('/content/train-w.txt', 'r')
  #text = f.read()
  #return text

#df = df.dropna()

#df2= pd.concat(frames)
#df2['ArticleText'] = df2.loc[0].apply(lambda x: getArticleText(x))
#df2['ArticleText'] = df2['ArticleText'].apply(lambda x: re.sub(r'(\n)+', '. ', str(x)))


def cleanQuestion(text):
  text = str(text)
  wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  return " ".join([word for word in words])

def cleanAnswer(text):
  text = str(text)
  wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  return " ".join([word for word in words])

def cleanText(text):
  text = str(text)
  wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s\.\?]', '', text).split()
  return " ".join([word for word in words])

#df['Question'] = df['Question'].apply(lambda x: cleanQuestion(x))
#df['Answer'] = df['Answer'].apply(lambda x: cleanAnswer(x))
#df['ArticleText'] = df['ArticleText'].apply(lambda x: cleanText(x))


In [ ]:
print(df1.columns.tolist())

['Question', ' Answer']


In [ ]:
df1[' Answer']

0                           အသိပညာသည် စွမ်းအားဖြစ်သည်။ 
1        အဲ့ အသိပညာကိုတကယ်အသုံးချမှ စွမ်းအားဖြစ်လာတာပါ။
2     အကျိုးရှိတယ်ဆိုတာ ငွေကြေးအမြတ်အစွန်းသာ ဆိုလိုခ...
3     သင်၏ဉာဏ်ရည်ဉာဏ်သွေး၊ သင်၏ဩဇာလွှမ်းမိုးမှု၊ သင့...
4     စာတစ်ခုခုကို ဖတ်လိုက်တာကို စိတ်ငြိမ်ဆေးအဖြစ် သ...
5     ပညာရှင်တစ်ယောက်ဟာ ဆယ်စုနှစ်ပေါင်းများစွာ အတွေ့...
6     ၎င်းသည် စိုးရိမ်ပူပန်မှု၊ လွန်ကဲသော ဖိစီးမှုကိ...
7      ဒါပေမယ့် မြန်မြန်ဖတ်ရတာ မဟုတ်ဘဲ တကယ်နားလည်သဘေ...
8     ပြင်းထန်တဲ့အရှိန်နဲ့ သတင်းအချက်အလက် ပမာဏတွေကလည...
9     ာလို့လဲဆိုတော့ ခေါင်းဆောင်များသည် စာဖတ်သူများဖ...
10    Oprah Winfreyပဲဖြစ်ဖြစ်Tony Robbinsပဲဖြစ်ဖြစ် ...
11    သင့်အသက်အရွယ် သို့မဟုတ် သင့်နောက်ခံ၊ သင့်အသက်မ...
12                           F A S T နည်းလမ်းဖြစ်ပါသည်။
13    F(Forget) – ဘယ်စာဖတ်ခြင်းအလေ့အကျင့်ဟောင်းကို သ...
14    A(Active)- စာဖတ်ဖို့ ဘယ်အရာက သင့်ကို လှုံ့ဆော်...
15    S(State) – စာဖတ်တဲ့အခါ သင်ဟာ ဘယ်အခြေအနေမှာ ရော...
16    T(Teach) – ဒါကို တခြားတစ်ယောက်ကို ဘယ်လိုပြန်သင...
17    စာဖတ်သူတွေ မေ့ရတဲ့ အကြောင်းရင်းတွေထဲက တစ်ခ

In [ ]:
df2

,သင် ၏ ဉာဏ်ရည်ဉာဏ်သွေး ၊ သင် ၏ ဩဇာလွှမ်းမိုး မှု ၊ သင့် ဝင် ငွေ နှင့် သင့် အကျိုးသက်ရောက် မှု ကို လည်း မြှင့်တင် နိုင် စေ ရန် သင့် စိတ် ကို ကျွမ်းကျင် အောင် လုပ် ပါ ။,ArticleText
0,အခု မေးခွန်း လေး တွေ စ မေး ကြည့် ပါ မယ် ။ သင် ...,အခု မေးခွန်း လေး တွေ စ မေး ကြည့် ပါ မယ် ။ သင် ...
1,သင့် မှာ မ ဖတ် ရ သေး တဲ့ စာအုပ် တွေ ရှိ သလား ။...,သင့် မှာ မ ဖတ် ရ သေး တဲ့ စာအုပ် တွေ ရှိ သလား ။...
2,အဲ့လို မေ့ သွား တာ အရမ်း စိတ်ပျက် ဖို့ ကောင်း ...,အဲ့လို မေ့ သွား တာ အရမ်း စိတ်ပျက် ဖို့ ကောင်း ...
3,ဒါ မှ မ ဟုတ် သင့် အတွက် စာဖတ် တာ က ပင်ပန်း ရုံ...,ဒါ မှ မ ဟုတ် သင့် အတွက် စာဖတ် တာ က ပင်ပန်း ရုံ...
4,စာ တစ် ခု ခု ကို ဖတ် လိုက် တာ ကို စိတ် ငြိမ် ဆ...,စာ တစ် ခု ခု ကို ဖတ် လိုက် တာ ကို စိတ် ငြိမ် ဆ...
5,အဲဒါ က မင်း အတွက် မှန် တယ် ဆို ရင် ဒီ စာ ကို မ...,အဲဒါ က မင်း အတွက် မှန် တယ် ဆို ရင် ဒီ စာ ကို မ...
6,လူ တွေ က အချိန် အများဆုံး သုံး ပြီး ပုံမှန် လု...,လူ တွေ က အချိန် အများဆုံး သုံး ပြီး ပုံမှန် လု...
7,အသိပညာ သည် စွမ်းအား ဖြစ် သည် ။,အသိပညာ သည် စွမ်းအား ဖြစ် သည် ။
8,ဒါပေမယ့် အဲ့ အသိပညာ ကို တကယ် အသုံးချ မှ စွမ်းအ...,ဒါပေမယ့် အဲ့ အသိပညာ ကို တကယ် အသုံးချ မှ စွမ်းအ...
9,အသိပညာ သည် အကျိုး ရှိ ပါ သည် ။,အသိပညာ သည် အကျိုး ရှိ ပါ သည် ။


In [ ]:

dataset = []
#title = ""
for i in range(0,len(df1)):
    #this_title = df.iloc[i]['ArticleTitle']
    #if (this_title!=title):
        #title = this_title
        text = df2.loc[i]['ArticleText']
        splitted = text.split(sep='\n')
        for j in range(len(splitted)):
            text = splitted[j]
            if(text!=''):
                words = text.split()
                dataset.append(words)

dataset.append(df1.loc[i]['Question'].split())
dataset.append(df1.loc[i][' Answer'].split())



In [ ]:
dataset


[['အခု',
  'မေးခွန်း',
  'လေး',
  'တွေ',
  'စ',
  'မေး',
  'ကြည့်',
  'ပါ',
  'မယ်',
  '။',
  'သင်',
  'စာဖတ်',
  'စရာ',
  'တွေ',
  'အများကြီး',
  'ရှိ',
  'နေ',
  'ပြီး',
  'အချိန်',
  'အရမ်း',
  'နည်း',
  'လွန်း',
  'နေ',
  'ပါ',
  'သလား',
  '။'],
 ['သင့်',
  'မှာ',
  'မ',
  'ဖတ်',
  'ရ',
  'သေး',
  'တဲ့',
  'စာအုပ်',
  'တွေ',
  'ရှိ',
  'သလား',
  '။',
  'စာ',
  'တစ်',
  'ခု',
  'ခု',
  'ကို',
  'ဖတ်',
  'ပြီး',
  'သွား',
  'ပြီး',
  'ဆုံး',
  'သွား',
  'လျှင်',
  'ဖတ်',
  'နေ',
  'တဲ့',
  'အရာ',
  'ကို',
  'မေ့',
  'သွား',
  'သလား',
  '။'],
 ['အဲ့လို',
  'မေ့',
  'သွား',
  'တာ',
  'အရမ်း',
  'စိတ်ပျက်',
  'ဖို့',
  'ကောင်း',
  'ပါ',
  'တယ်',
  '။'],
 ['ဒါ',
  'မှ',
  'မ',
  'ဟုတ်',
  'သင့်',
  'အတွက်',
  'စာဖတ်',
  'တာ',
  'က',
  'ပင်ပန်း',
  'ရုံ',
  'ပဲ',
  '။'],
 ['စာ',
  'တစ်',
  'ခု',
  'ခု',
  'ကို',
  'ဖတ်',
  'လိုက်',
  'တာ',
  'ကို',
  'စိတ်',
  'ငြိမ်',
  'ဆေး',
  'အဖြစ်',
  'သုံး',
  'ပြီး',
  'အိပ်ပျော်',
  'စေ',
  'ရုံ',
  'ပဲ',
  'လုပ်',
  'မိ',
  'နေ',
  'လား',
  '?']

In [ ]:
model = gensim.models.Word2Vec(dataset, size=100, window=8, min_count=1, sg=1) # I have 8 cpu cores
# sg = {0, 1} – Training algorithm: 1 for skip-gram; otherwise CBOW



In [ ]:


def get_embedding(sentence):
  pos_sum = [0.0 for i in range(100)]
  num = 0
  words = sentence.split()
  for i in words:
    try:
      embed = model.wv[i]
    except:
      continue
    else:
      pos_sum += embed
      num +=1
  if(num==0):
    return pos_sum
  else:
    pos_sum /= num
    return pos_sum

def get_answer(question, answer_para):
  question_embedding = get_embedding(rem_stop(question))
  min_distance = math.inf
  answer = 0
  for i in range(len(answer_para)):
    answer_embedding = get_embedding(rem_stop(answer_para[i]))
    distance = np.linalg.norm(question_embedding-answer_embedding)
    if (distance < min_distance):
      answer = i
      # print(answer)
      min_distance = distance
  return answer_para[answer]

def rem_stop(sentence):
    strr=''
    my_string = sentence.split()
    for i in range(len(my_string)):
        if my_string[i] not in stopwords.words('english'):
            strr = strr+' '+my_string[i]
    return strr[1:]

def get_answer_cosine(question, answer_para):
  question_embedding = get_embedding(rem_stop(question))
  max_similarity = -math.inf
  answer = 0
  for i in range(len(answer_para)):
    answer_embedding = get_embedding(rem_stop(answer_para[i]))
    similarity = cosine_similarity(np.expand_dims(question_embedding,0), np.expand_dims(answer_embedding,0))
    if (similarity > max_similarity):
      answer = i
      max_similarity = similarity
  return answer_para[answer]



In [ ]:

index = 10
my_text = df2.loc[index]['ArticleText']
#temp_sentences = my_text.split(sep='.')
sentences=[]
for i in range(len(temp_sentences)):
    if(temp_sentences[i]!=''):
        sentences.append(temp_sentences[i])
my_question = df1.loc[index]['Question']



In [ ]:

print(my_question) # Actual Question
#print(rem_stop(my_question)) # Answer without stopwords
print(df.iloc[index][' Answer']) # Actual Answer



ဘယ်သူတွေကစာဖတ်ဝါသနာပါသူတွေလဲ။
Oprah Winfreyပဲဖြစ်ဖြစ်Tony Robbinsပဲဖြစ်ဖြစ် Bill Gates ပဲဖြစ်ဖြစ်၊ လူ့အဖွဲ့အစည်းမှာ တကယ်ကို လှုပ်ရှားနေတဲ့သူတို့သည် စာဖတ်ဝါသနာပါသူများဖြစ်သည်။


In [ ]:
print(get_answer(my_question, sentences)) # Our model's prediction using euclidean distance
print("\n")
print(get_answer_cosine(my_question, sentences))

အကျိုး ရှိ တယ် ဆို တာ ငွေကြေး အမြတ်အစွန်း သာ ဆို လို ချင် တာ မ ဟုတ် ပါ ။


အကျိုး ရှိ တယ် ဆို တာ ငွေကြေး အမြတ်အစွန်း သာ ဆို လို ချင် တာ မ ဟုတ် ပါ ။


In [ ]:
words = list(model.wv.vocab)
print(words)

['အခု', 'မေးခွန်း', 'လေး', 'တွေ', 'စ', 'မေး', 'ကြည့်', 'ပါ', 'မယ်', '။', 'သင်', 'စာဖတ်', 'စရာ', 'အများကြီး', 'ရှိ', 'နေ', 'ပြီး', 'အချိန်', 'အရမ်း', 'နည်း', 'လွန်း', 'သလား', 'သင့်', 'မှာ', 'မ', 'ဖတ်', 'ရ', 'သေး', 'တဲ့', 'စာအုပ်', 'စာ', 'တစ်', 'ခု', 'ကို', 'သွား', 'ဆုံး', 'လျှင်', 'အရာ', 'မေ့', 'အဲ့လို', 'တာ', 'စိတ်ပျက်', 'ဖို့', 'ကောင်း', 'တယ်', 'ဒါ', 'မှ', 'ဟုတ်', 'အတွက်', 'က', 'ပင်ပန်း', 'ရုံ', 'ပဲ', 'လိုက်', 'စိတ်', 'ငြိမ်', 'ဆေး', 'အဖြစ်', 'သုံး', 'အိပ်ပျော်', 'စေ', 'လုပ်', 'မိ', 'လား', '?', 'အဲဒါ', 'မင်း', 'မှန်', 'ဆို', 'ရင်', 'ဒီ', 'ဖြစ်', 'ဆက်', 'လူ', 'အများဆုံး', 'ပုံမှန်', 'သတင်း', 'အချက်အလက်', 'စီမံ', 'အသိပညာ', 'သည်', 'စွမ်းအား', 'ဒါပေမယ့်', 'အဲ့', 'တကယ်', 'အသုံးချ', 'လာ', 'အကျိုး', 'ငွေကြေး', 'အမြတ်အစွန်း', 'သာ', 'လို', 'ချင်', 'ပို', 'မြန်မြန်', 'သင်ယူ', 'နိုင်', 'လေ', 'ဝင်', 'ငွေ', 'ပညာရှင်', 'ယောက်', 'ဟာ', 'ဆယ်စုနှစ်', 'ပေါင်း', 'များ', 'စွာ', 'အတွေ့အကြုံ', 'သူ', 'အုပ်', 'ထဲ', 'ထည့်', 'ရေး', 'ရက်', 'အနည်းငယ်', 'အတွင်း', 'ထိုင်', 'အဲဒီ', 'ကာ', 'သိ', 'ဘဲ', 'နားလည်', 'သဘောပ

In [ ]:
filename= model.save('model.bin')


INFO:gensim.utils:saving Word2Vec object under model.bin, separately None
2023-03-15 23:39:38.958 saving Word2Vec object under model.bin, separately None
INFO:gensim.utils:not storing attribute vectors_norm
2023-03-15 23:39:38.961 not storing attribute vectors_norm
INFO:gensim.utils:not storing attribute cum_table
2023-03-15 23:39:38.963 not storing attribute cum_table
INFO:gensim.utils:saved model.bin
2023-03-15 23:39:38.970 saved model.bin


In [ ]:
model.wv.save_word2vec_format('model.txt', binary=False)

INFO:gensim.models.utils_any2vec:storing 260x100 projection weights into model.txt
2023-03-15 23:20:23.956 storing 260x100 projection weights into model.txt
